 # Practice  Soft SVM.
 ----
  
  University : UTEC \\
  Course       : Machine Learning \\
  Professor    : Cristian López Del Alamo \\
  Topic        : Soft SVM \\
  Termina      : 12:45
   

 ----

Write the names and surnames of the members and the percentage of participation of each one in the development of the practice:
 - Integrante 1: (%)
 - Integrante 2: (%)
 - Integrante 3: (%)
 - Integrante 4: (%)


 ----


  




In [197]:
# Loading Libraries

import numpy as np
import math
import pandas as pd
import random
import matplotlib.pyplot as plt
import cvxopt
import seaborn as sns
import itertools
import pywt
import os
from sklearn.metrics import confusion_matrix
import zipfile
from skimage import io, color

## Normalization

In [198]:
# Data Normalizing
def normalizacion(data):
    data = (data - data.min()) / (data.max() - data.min())
    return data
 # write your code here

## 1. Feature Vector

A wavelet is a more efficient method than Fourier transforms for approximating a non-periodic function. Since an image can be seen as a two-dimensional function in a 3D space, we can approximate it using wavelets. In this case, we use the Hard Wavelet. The goal is to obtain a characteristic vector that represents each image

----
Parámetros de la Función **haar**
- f: Image File Name
- n: Number of slider in the image

In [199]:
# Wavelet function for extracting image characteristics. This function returns a vector with image characteristics
def haar(f, n):
  imagen = io.imread(f)
  for i in range(n):
    imagen, (LH, HL, HH) = pywt.dwt2(imagen, 'haar')
  gfg = np.array(imagen)
  return gfg.flatten()

### Encode Function
Encode is a function that encodes images into feature vectors using wavelets to extract relevant information from each image. Encode takes the path where the images are located, encodes them, and returns a matrix where the ith element of the matrix is the feature vector of image i.

In [200]:
# This function loads all images in the specified path and returns a vector containing the feature vectors of these images. In the example, we use three splits.
def encode(path, cuts):
  encodings = []
  for filename in os.listdir(path):
      image_file = os.path.join(path, filename)
      if os.path.isfile(image_file):
          encodings.append(haar(image_file, cuts))
  return encodings

## Loading the data and extracting features using Hard Wavelet

### get_data Function
This function takes as input the paths where images of happy and disgust people are located, encodes them, and stores them in matrices. One matrix contains rows with features of happy people and another matrix contains features of sad people; both are combined into the same matrix. The rows of the matrix are randomized to then create the database that will train our soft SVM

In [201]:

def get_data(emotion_1, emotion_2, cuts):
  data  =  np.array(encode(emotion_1,cuts))
  data  = np.insert(data, 0, 1, axis=1)
  temp =  np.array(encode(emotion_2,cuts))
  temp = np.insert(temp, 0, -1, axis=1)
  data = np.concatenate((data, temp), axis=0)
  for i in range(10):
    np.random.shuffle(data)
  y = data[:,0]
  x = data[:, 1:]
  return x, y

# Load train and test data

We mount the drive in Colab, get the path where our images are located, and load the test and training data

[Download DataSet](https://drive.google.com/drive/folders/1TvgyPE5l6TWtx7Fa6ZSKVUG2SsvEvh7l?usp=sharing)

In [202]:
# from google.colab import drive
# drive.mount('/content/drive')

path = '.'

c1_emotion = 'fear'
c2_emotion = 'neutral'

c1_emotion_train  = path +  '/train/' + c1_emotion
c2_emotion_train  = path +  '/train/' + c2_emotion

c1_emotion_test  = path +  '/validation/' + c1_emotion
c2_emotion_test  = path +  '/validation/' + c2_emotion

train_x, train_y = get_data(c1_emotion_train, c2_emotion_train,3)
test_x, test_y = get_data(c1_emotion_test, c2_emotion_test,3)

### 1. Hypothesis :  Remember, you can achieve it using linear algebra.

$h(x_i) = x_i*w^t + b$

In [203]:
def h(X,w,b):
    return np.sign(np.dot(X,w)+b)


### 2. Loss function

$L = \frac{1}{2}\parallel W \parallel_2^2  + C\sum_{i=0}^nmax(0,1-y_i(x_iw^t + b)) $


In [204]:
# Implemente la función de pérdida del soft SVM
def loss(y, y_aprox, C, W):
    return 0.5*np.dot(W, W) + C*np.sum(np.maximum(0,1-y*y_aprox))

### 3. Derivatives

1. iF ($y_i*h(x_i)) \lt 1 $
 - $\frac{\partial L}{\partial w} = w + C\sum_{
 i=0}^n-y_i*x_i $
2. else
 - $\frac{\partial L}{\partial w} = w $

----
**Don't forget to find the derivatives for Bias.**







In [205]:
# Implemente la función para obtener las derivadas de W
def derivatives(x, y, y_aprox, w, C):
  print("Y: ",y)
  print("Y aprox:", y_aprox)
  print("Y SHAPE: ", y.shape)
  print("Y APROX SHAPE: ", y_aprox.shape)
  #get the scalar value of y and y_aprox
  res = np.dot(y, y_aprox)
  print("RES: ", res)
  if res < 1:
    dw = w - C*y*x
    db = -C*y
  else:
    dw = w
    db = 0
  return dw, db

### 4. Change Parameters


1. iF ($y_i*h(x_i)) \lt 1 $
 - $w  = w -  \alpha( C\sum_{
 i=0}^n-y_i*x_i) $
2. else
 - $w = w -  \alpha*w$

----
**Don't forget charge the Bias.**


In [206]:
def Update(y,y_aprox,w, b, db, dw, alpha):
    res = np.dot(y, y_aprox)
    if res < 1:
        w = w - alpha*(dw)
        b = b - alpha*(db)
    else:
        w = w - alpha*(w)
        b = b - alpha*(b)
    return w,b

# 5. Training stage

In [207]:

def training(X, Y, C, alpha, epochs):
  w = np.array([np.random.rand() for i in range(X.shape[1])])
  b = np.random.rand()
  error = []
  for i in range(epochs):
    Y_aprox = h(X,w,b)
    db, dw = derivatives(X, Y,Y_aprox, w, C)
    w, b = Update(Y,Y_aprox,w, b, db, dw, alpha)
    L = loss(Y,Y_aprox,C, w)
    error.append(L)
  return w, b, error

### 6. Testing Stage

In [208]:
# Implemente la función de testing
def testing(X,W,b):
  y_aprox = []
  # write your code here
  print("X SHAPE: ", X.shape)
  for i in range(X.shape[0]):
    y_aprox = np.sign(np.dot(X[i], W.transpose())+b)
  return np.array(y_aprox)

In [209]:
# Main Program

m = train_y.size
k = train_x[0].size
train_x_norm = np.apply_along_axis(normalizacion, 1, train_x)
test_x_norm = np.apply_along_axis(normalizacion, 1, test_x)

W, b, e1 = training(train_x_norm, train_y, 1e6, 1e-8, 1200)
m = test_y.size
y_pred = testing(test_x_norm, W, b)
test_y = test_y.astype('int')

correct = 0
correct = np.sum(y_pred == test_y)

print("Clasificados correctamente:", correct)
print("Clasificados incorrectamente:", len(test_y)-correct)
print("% de efectividad", round(100*correct/len(test_y), 2))


C:\Users\vilch\AppData\Local\Temp\ipykernel_33824\3971057755.py:3: RuntimeWarning: invalid value encountered in divide
  data = (data - data.min()) / (data.max() - data.min())


Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1

Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1. 1. ... 1. 1. 1.]
Y SHAPE:  (9085,)
Y APROX SHAPE:  (9085,)
RES:  nan
Y:  [ 1. -1.  1. ... -1. -1. -1.]
Y aprox: [1. 1

In [212]:
matrix = confusion_matrix(test_y, y_pred)
df2 = pd.DataFrame(matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis], index=["Angry", 'Happy'], columns=["Angry", 'Happy'])
sns.heatmap(df2, annot=True, cbar=None, cmap="Greens")
plt.title("Confusion Matrix"), plt.tight_layout()
plt.xlabel("Predicted")
plt.ylabel("Real")
plt.show()

(2234,) ()


TypeError: Singleton array array(1.) cannot be considered a valid collection.